In [22]:
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,3)
%config InlineBackend.figure_format='retina'

# How to Finally Ditch Spreadsheets in favour of Notebooks

Recently, it is becoming more and more common to hear stories where excel seems to behave like it is haunted by some kind of ghosts: [human genes names are mistaken as dates ](https://www.theverge.com/2020/8/6/21355674/human-genes-rename-microsoft-excel-misreading-dates), 
limits on the maximum number of rows simply [delete additional rows without warning the user](https://www.theguardian.com/politics/2020/oct/05/how-excel-may-have-caused-loss-of-16000-covid-tests-in-england), 
and it also happened that [someone lost around **6B$** for a spreadsheet error](https://www.businessinsider.com/excel-partly-to-blame-for-trading-loss-2013-2?IR=T).

<img src='https://cdn.vox-cdn.com/thumbor/FmKvD1XuFdjxrvq2DBK9s0ZB3Ds=/1600x0/filters:no_upscale()/cdn.vox-cdn.com/uploads/chorus_asset/file/21700662/excel_gene_names_111.gif' height='200px'>

## Introduction

What it's slowly coming out is that when dealing with the so-called *big data*, AKA huge collections of data, usually containing more than 10K rows, spreadsheets are not the tool to go.
There are several limitations in many aspects, like the maximum number of rows and columns, auto-formatting which can lead to mistakes and, of course, 
since spreadsheet is extremely widespread, different users having different literacies uses them, creating unpredictables situations, 
especially when a situation escalates and the tool is the only tool used.

<img src='./excel-limits.png' height='200px'>

In this article, I would like to show how to gradually switch from the spreadsheet in favour of modern notebooks, 
cloud storage, and how to performs most common spreadsheet operations using Python along with Pandas on a pretty heavy dataset.

Nowadays, there are many solutions out there that allow doing this: you can install [Jupyter](https://jupyter.org/),
work on [Colab](https://colab.research.google.com/) or use some hosted tools such as [Sagemaker](https://aws.amazon.com/sagemaker/). 
However, it is not always so straightforward to get started and plug everything together. That is the reason why, for this article, I will use [Deepnote](https://deepnote.com/), a free, hosted, and plug-n-play 
service where you can run notebooks, have real-time collaboration, and natively access different databases.

## The data
The dataset used in this article is the [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/olistbr/brazilian-ecommerce), 
it contains information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. 
Its features allow viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers.

Since this dataset consists of nine different tables in `.csv` format it is helpful to have a schema of the dataset, AKA a *map* that helps us to understand 
how tables are related to each other, giving the possibility to perform a different operation such as `VLOOKUP`, `SUMIF`, `CONCATENATE` and many others.

You can think of it as having different sheets related to each other.
<img src='./data-schema.png' height='400px'>

<!-- ## Loading the data
The tables are hosted on AWS S3, however, Deepnote has an extremely powerful integration to load data from S3, 
it is enough to add the integration, insert your bucket name and credentials, and *voit-la*, data available!
<img src='./s3-card.png' height='150px'> -->

## Loading the data
The tables can be uploaded into deepnote and are ready to be accesses, you can check that the data is correctly uploaded by executing the `ls` command:

In [23]:
!ls e-commerce-data

olist_customers_dataset.csv	  olist_orders_dataset.csv
olist_geolocation_dataset.csv	  olist_products_dataset.csv
olist_order_items_dataset.csv	  olist_sellers_dataset.csv
olist_order_payments_dataset.csv  product_category_name_translation.csv
olist_order_reviews_dataset.csv


## Answering business questions
In this section, a set business questions related to most the common KPIs and insights for an e-commerce are shown along with the code/solution to achieve it.
More precisely, we will answer question related to:

- Total value of sales by the hour, day, week, or month.
- Average value of transaction.
- Where customers are based: it can be possible to identify where most customers are based and which are the most profitable cities.
- Most ordered products.


### Total value of sales
The goal of this question is to find out the total value of sales by the hour, day, week, and month.
It can be answered by grouping orders by the desired dimension and then the `SUM` function can be applied.

To answer this question we will need access to the timestamp (`order_purchase_timestamp`) when an order is created, and the value of the order. 
We can achieve this by using two different tables: `olist_order_payments_dataset` and `olist_orders_dataset`, according to the database schema
the two tables can be joined together using the `order_id` column. Let's do this!

In [3]:
# load tables
orders = pd.read_csv('./e-commerce-data/olist_orders_dataset.csv')
orders_payments = pd.read_csv('./e-commerce-data/olist_order_payments_dataset.csv')

In [4]:
orders.info()

In [5]:
orders_payments.info()

However, after inspecting the dataset, I noticed the there is a column named `payment_installments`, meaning that the values in the `payment_value`
column are not reflecting the real order value. To have the real order value, we will have to multiply the number of instalments with the payment value. 

In [6]:
# merged tables and convert timestamp to pandas-like format
orders_merged = orders.merge(orders_payments, how='inner', on='order_id')
orders_merged['order_purchase_timestamp'] = pd.to_datetime(orders_merged['order_purchase_timestamp'])

In [7]:
# multiply the number of installments with the payment value
orders_merged['order_value'] = orders_merged['payment_value'] * orders_merged['payment_installments']

In [8]:
# group by dimension and plot

orders_merged.groupby(orders_merged['order_purchase_timestamp'].dt.day)['order_value'].sum().plot()
plt.title('Sales by day')

In [9]:
orders_merged.groupby(orders_merged['order_purchase_timestamp'].dt.week)['order_value'].sum().plot()

plt.title('Sales by week')

In [10]:
orders_merged.groupby(orders_merged['order_purchase_timestamp'].dt.month)['order_value'].sum().plot()
plt.title('Sales by month')

### Average value of a transaction
This KPI gives insights about how a typical shopping basked value evolves: from this is possible to understand if customers
tend to buy multiple items in one order or rather have many small transactions. It is helpful when, for example, when architecting some promotions or discounts.

To achieve this we can use a strategy similar to the previous one: we will group transaction value by the month and get the average.

In [11]:
print('Average value of order by month')
avg_value = orders_merged.groupby(orders_merged['order_purchase_timestamp'].dt.month)['order_value'].mean().reset_index()
avg_value.columns = ['Month', 'Value']

avg_value = avg_value.round(2)
avg_value['Month'] = pd.to_datetime(avg_value['Month'], format='%m').dt.month_name().str.slice(stop=3)

avg_value

Deepnote offers a really cool feature that lets you automatically plot data from a dataframe, just press on the "visualize" table inside the output section of a cell:

In [12]:
avg_value

### Where customers are based
Knowing the location where a customer is based open the gates for many things: it is possible to understand which areas are more profitable than other, 
where our customers are based, and which regions/cities tend to spend more (or less) money on our platform.

To get the data ready we can, again, have a look at the schema: the `olist_customers_dataset` 
table contains information customers geography. We can use this table and do some `groupby` along with `value_counts:`

In [13]:
customers = pd.read_csv('e-commerce-data/olist_customers_dataset.csv')
customers.head()

In [14]:
customers['customer_city'].value_counts().to_frame()

In [15]:
orders = pd.read_csv('e-commerce-data/olist_orders_dataset.csv')
orders.info()

print('\n')

order_items = pd.read_csv('e-commerce-data/olist_order_items_dataset.csv')
order_items.info()

### Most popular ordered products
To get this view it is necessary to merge the `olist_orders_dataset` with the `olist_order_items_dataset`, 
the two tables can be joined using the `order_id` column. 
Then, similarly as before, it can be possible to count values of the `product_id` column. 
However, to get the item category, another join operation is needed: the item category name is stored inside the `olist_products_dataset`, 
which can be joined with `olist_order_items_dataset` using the `product_id` column.

In [16]:
orders_merged_items = orders.merge(order_items, how='inner', on='order_id')[['product_id']]
orders_merged_items.head()

In [17]:
product_desc = pd.read_csv('e-commerce-data/olist_products_dataset.csv')
product_desc.head()

In [18]:
# join orders_merged_items with product_desc on product_id column
product_count = orders_merged_items.merge(product_desc, how='inner', on='product_id')
product_count.head()

Everything seems to work well, however, product category names are in Portuguese, and not in English. 
Luckily the dataset comes with another table `product_category_name_translation` which, for every category name, the translated name in English. We can substitute the Portuguese column with the English one:

In [19]:
eng_cat = pd.read_csv('e-commerce-data/product_category_name_translation.csv')
eng_cat.head()

In [20]:
product_count = product_count.merge(eng_cat)

It is now possible to apply the `value_counts` function and plot the 20 most common categories:

In [21]:
product_count['product_category_name_english'].value_counts()[0:20].plot(kind='bar')

## Where to go now?
In this short guide, I wanted to show some basic operation that could help you switch from the old-fashioned spreadsheet to
notebooks. Additional examples, tips and guides can be found in the [pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html#intro-to-pandas)
documentation. 

Regarding this dataset, in particular, I want to give you some starting points where you could expand this tutorial, for example, 
you could use the the `olist_geolocation_dataset` along with the joined `olist_customers_dataset` and ` olist_orders_dataset` to create some heatmaps using [`gmaps`](https://jupyter-gmaps.readthedocs.io/en/v0.3.2/gmaps.html)
displaying, for example, where the highest number of orders comes from, or where the average basket value is higher.

Moreover, another good idea could be to create some sort of clustering model for customers, using variables such as the city a customer
comes from, the number of orders, the category of products the customers buys, and many more. Have a look at [this](https://levelup.gitconnected.com/unsupervised-learning-how-to-categorize-an-unlabelled-dataset-480fa2fdddd9) story to get ad idea.

Feel free to copy this notebook, run it, or change it!
[<img src='https://deepnote.com/buttons/launch-in-deepnote.svg'/>](https://deepnote.com/launch?template=deepnote&url=https%3A%2F%2Fgithub.com%2Frobertosannazzaro%2Fditch-spreadsheets-for-notebooks)